# Astropy
Astropy is a package dedicated to Astronomy and Astrophysics.

In [ ]:
import matplotlib.pyplot as plt

## Constants and Equations

In [ ]:
# Importing Essential Packages
from astropy import constants as const # Universal Constants
from astropy import units as u # Units (unit conversion)

In [ ]:
print(const.G)
print('\n')
print(const.c)
print('\n')
print(const.c.to('km/s'))

Recall that:
$$
    F = \frac{GMm}{r^2}
$$

In [ ]:
# Compute force due to gravity of Sun on Earth
F = (const.G * const.M_sun * const.M_earth) / (const.au) ** 2 
# Give force in Newtons
print(F.to(u.N)) 
print(F) 
print(F.unit)

In [ ]:
# We can assign types with multiplication
list = [1, 2, 3]
listInM = list * u.m
print(listInM)
print(listInM.unit)
print(listInM.to(u.mm))

# TODO: Compute k
Recall Kepler's Third Law of Planetary Motion:
$$
    \frac{r^3}{T^2} = k
$$
Give your answer is `pc^3 / yr^2`

In [ ]:
# Put your code here

## Reading & Writing File
.fits files are a common way to store data in astronomy

In [ ]:
# Importing Essential Packages
from astropy.table import Table # Astropy works with .fits files

In [ ]:
glh = Table.read("data/galah_dr4_trimmed.fits", format="fits", hdu=1)
glhxlim

In [ ]:
# Write the data
glh.write('data/galah_dr4_trimmed_new.fits', format='fits')

In [ ]:
glh.colnames

In [ ]:
plt.plot(glh['teff'], glh['logg'], 'x', markersize=0.2)

plt.title("HR Diagram")
plt.xlabel("teff")
plt.ylabel("logg")
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

## Models & Fitting

In [ ]:
import numpy as np
from astropy.modeling import models, fitting

In [ ]:
# Fitting a Gaussian to existing data
tmp = glh[glh['logg'] < 3]
plt.plot(tmp['teff'], tmp['logg'], 'x', markersize=0.2)

plt.title("HR Diagram")
plt.xlabel("teff")
plt.ylabel("logg")
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

In [ ]:
# Line of Best Fit
model = models.Linear1D()
fitter = fitting.LinearLSQFitter()
best_fit = fitter(model, tmp['teff'], tmp['logg'])
print(best_fit)

In [ ]:
plt.plot(tmp['teff'], tmp['logg'], 'x', markersize=0.2)
plt.plot(tmp['teff'], best_fit(tmp['teff']))

plt.title("HR Diagram")
plt.xlabel("teff")
plt.ylabel("logg")
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

In [ ]:
# Polynomial of Best Fit
odel_poly = models.Polynomial1D(degree=3)
fitter_poly = fitting.LinearLSQFitter() 
best_fit_poly = fitter_poly(odel_poly, tmp['teff'], tmp['logg'])
print(best_fit_poly)

In [ ]:
plt.plot(tmp['teff'], tmp['logg'], 'x', markersize=0.2)
plt.plot(tmp['teff'], best_fit_poly(tmp['teff']), 'r+')

plt.title("HR Diagram")
plt.xlabel("teff")
plt.ylabel("logg")
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

In [ ]:
# Gaussian
mu, sigma, amplitude = 0.0, 10.0, 10.0
N = 100
x = np.linspace(-30, 30, N)
y = amplitude * np.exp(-(x-mu)**2 / (2*sigma**2))
y = np.array([y_point + np.random.normal(0, 1) for y_point in y])   #Another way to add random gaussian noise
sigma = 1

In [ ]:
model_gauss = models.Gaussian1D()
fitter_gauss = fitting.LevMarLSQFitter()
best_fit_gauss = fitter_gauss(model_gauss, x, y)

plt.plot(x, y, 'r+')
plt.plot(x, best_fit_gauss(x), 'g-', linewidth=1, label='astropy.modeling')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.legend()

# TODO: Try to fit x and y with a polynomial regressor

In [ ]:
# Put your code here

## Bonus Astropy Functionality Worth Exploring
- Look into Chi-squared Test for assessing fit
    - Need to include uncertainty as an additonal parameter

# Gala 
Gala is used to explore the orbits of Stars
This guide is based on: [Computing Galactic Orbits of Stars with Gala](https://learn.astropy.org/tutorials/gaia-galactic-orbits.html)

In [ ]:
# Importing Essential Packages
from astropy.table import Table # Astropy works with .fits files
import matplotlib.pyplot as plt 
from astropy import units as u # Units (unit conversion)

In [ ]:
# 4096 random stars within 100 pc (or with a parallax > 10 mas) of the Sun
gaia_data = Table.read("data/gaia_data.fits", format="fits", hdu=1)

In [ ]:
gaia_data

In [ ]:
# Position of stars
plt.plot(gaia_data['ra'], gaia_data['dec'], 'r+')

In [ ]:
# Velocity + Radial Velocity of Stars
plt.scatter(gaia_data['pmra'], gaia_data['pmdec'], c=gaia_data['radial_velocity'], marker='x', cmap=plt.cm.inferno, s=1)
cbar = plt.colorbar()
plt.xlabel("Proper Motion: Right Ascension")
plt.ylabel("Proper Motion: Declination")
cbar.set_label("Radial Velocity")

## Determine Galactic Stellar Orbits with Gala
### Converting Data Into Correct Coordinates

In [ ]:
# Import Essential Packages
import gala.potential as gp
import gala.dynamics as gd
import astropy.coordinates as coord
import numpy as np

In [ ]:
dist = coord.Distance(parallax=u.Quantity(gaia_data['parallax']))
# Furthest and closest star
dist.min(), dist.max()

In [ ]:
# Converts helocentric, spherical values to Galatocentric, Cartesian values
c = coord.SkyCoord(ra=gaia_data['ra'], 
                   dec=gaia_data['dec'],
                   distance=dist,
                   pm_ra_cosdec=gaia_data['pmra'], 
                   pm_dec=gaia_data['pmdec'],
                   radial_velocity=gaia_data['radial_velocity'])

In [ ]:
# Gets the parameters that we need to shift c by
coord.Galactocentric()

In [ ]:
# c is coordinates centered about centre of the Solar System
# transform moves to be at the centre of the Milkway
galcen = c.transform_to(coord.Galactocentric(z_sun=0*u.pc, galcen_distance=8.1*u.kpc))

In [ ]:
plt.plot(galcen.v_x.value, galcen.v_y.value, marker='.', linestyle='none', alpha=0.5)

plt.xlim(-125, 125)
plt.ylim(200-125, 200+125)


plt.xlabel('$v_x$ [{0:latex_inline}]'.format(u.km/u.s))
plt.ylabel('$v_y$ [{0:latex_inline}]'.format(u.km/u.s))

### Integrate Galactic Orbits

In [ ]:
milky_way = gp.MilkyWayPotential()
milky_way
# We can could use a different disk mass to compute 
# different_disk_potential = gp.MilkyWayPotential(disk=dict(m=8e10*u.Msun))
# different_disk_potential

In [ ]:
H = gp.Hamiltonian(milky_way)

In [ ]:
# Integrate orbits with H.integrate_orbit
# Compute where the star lies on Phase Space Diagram
w0 = gd.PhaseSpacePosition(galcen.cartesian)
# Determine Orbit using Hamiltonian (increment by 1 Mega-Years, between 0 and 500 Mega-years)
orbit = H.integrate_orbit(w0, dt=1*u.Myr, t1=0*u.Myr, t2=500*u.Myr)

In [ ]:
orbit.shape
# 500 rows (each star has 500 simulated phase space position)
# 4096 columns (each star has a column)

In [ ]:
# Orbital Parameters
E = orbit.energy()
L = orbit.angular_momentum()
L.shape
# Shape represents (cartesian coordinates e.g. x, y, z, #simulations, which star)

In [ ]:
# Energy of the first star in each simulation step
E[:, 0]

In [ ]:
# Angular Momentum in each simulation step
L[0, :, 0] # x direction
L[1, :, 0] # y direction
L[2, :, 0] # z direction

In [ ]:
# If your Juypter Notebook shows [ * ], it means the kernel is busy, give it a few seconds :)
orbit[:, 0].plot() # [row, col], [:, 0] = get all rows in first column

In [ ]:
orbit[:, 0].cylindrical.plot(['rho', 'z'])

# TODO: Graph the orbit of another star 
- Note each star is a column in orbit
   - Plot x vs y 
   - Plot rho vs z

Read the [documentation](http://gala.adrian.pw/en/latest/tutorials.html)